In [ ]:
## If using Google colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
import pandas as pd
import io
import numpy as np
# test

In [ ]:
# Set up read path, read file, write path, write file
curr_project_name = "project_erudition_emails_2"
notebook_path = "/mnt/c/Users/isaac/Desktop/mturks_assistant_github/mturks_assistant/notebooks"

input_filename='Batch_4515857_batch_results.xlsx'
read_path = notebook_path+"/../projects/"+curr_project_name+"/2_mOut_dirs/"+input_filename

output_filename = 'blast2_uni_directories_mod.xlsx'
write_path = notebook_path+"/../projects/"+curr_project_name+"/3_notebookProcessed_dirs/"+output_filename

## Read in MTurks dataframe and do preliminary reformatting and unstacking of data

In [ ]:
df=pd.read_excel(read_path)


# Clean NANs
print('df nan values: ',df.isnull().sum().sum())
df = df.fillna('N/A')
print('df nan values: ',df.isnull().sum().sum())

# Unstack the data side by side 
df['copy_id'] = -1

dfgroups = df.groupby(['Input.university_name','Input.city']).size().reset_index().rename(columns={0:'count'})

for i, x in dfgroups.iterrows():
  uni = x['Input.university_name']
  city = x['Input.city']
  df_zoom = df.loc[(df['Input.university_name']==uni) & (df['Input.city'] == city)]
    
  ctr = 1
  for j, y in df_zoom.iterrows():
    df.at[j,'copy_id'] = ctr
    ctr += 1

df2 = df.set_index(['Input.university_name','Input.city','copy_id'])
df2 = df2.unstack('copy_id')


# Clean NANs
print('df2 nan values: ',df2.isnull().sum().sum())
df2 = df2.fillna('N/A')
print('df2 nan values: ',df2.isnull().sum().sum())

# Remove all unnecessary columns for coming analysis
# KEEP ALL INPUT DATA FOR LATER CHECKING (all inputs, all answers, hitid, workerid)
df3 = df2[['HITId','WorkerId','Input.country','Input.university_name','Input.city','Answer.studentOrgList_url']]

# Clean NANs
print('df3 nan values: ',df3.isnull().sum().sum())
df3 = df3.fillna('N/A')
print('df3 nan values: ',df3.isnull().sum().sum())

## Selecting the winning data and making flags for you to manually check

In [ ]:
# Idea: you should confirm that the domain of directory link is what you are expecting...how? But all university website are formatted differently, some use external domains for their club directories.
# You must be able to quickly and automatically reject HITs if they are not good. Someone took advantage of you. It's the same worker. They could do this because you are basically approving everyone.
## Make a worker dataframe. 
# For each worker: 
# (1 count how many HITs they do) 
# (2 confirm they are not using the same link on more than one submission. Flag them if so. You will reject all of their HITs and block them.) 
# (3 for each HIT done by the worker, check the initial rating of the HIT (using my simple if/else method). Get the cumulative ratings of the HITs and compare to max possible rating, assign this worker rating to the worker. Use this worker trustworthiness rating to re-weight the redundant HITs, see if this forces a different automatic outcome. ) -- is this is a recursive thing? It reminds me of network eigencentrality ratings.
# (4 Assemble a global MTurk trustworthiness database based on all my projects. Blacklisted Turks, starred Turks.)

# You should include this information to the output you write and will manually check later. It makes it easier to trace back to find out what happened better. It's possible but slow and difficult to trace through everything
# In addition to what you already have, write the worker ID, the cumulative input info you gave them from the beginning of the process until now, and the link from the original spreadsheet where the HITs were stored (?) idk what this last thing is

# Also, you should make this for an arbitrary level of redundancy 

In [ ]:
redundancy = 3 # the redundancy is built into this it will take alot of work to automate this for more or less redundancy
item = 'Answer.student_org_url'
anchor = '.'

for index, row in df3.iterrows():
  # print("index ",index)
  # print("row item")
  # print(row['HITId'][1])

  
  # Case 1: All 3 are the same
  #if (row['1'+infix+item] == row['2'+infix+item] and row['1'+infix+item] == row['3'+infix+item]):
  if (row[item][1] == row[item][2] and row[item][1] == row[item][3]):
    #if ( anchor in row['1'+infix+item]):
    if ( anchor in row[item][1]):
      #row['Winner_'+item] = row['1'+infix+item]
      df3.loc[index, ('results','winner_'+item)] = row[item][1]
      #row[item+'flag'] = 'All good, triple consensus'
      df3.loc[index,('results','flag_'+item)] = 'All good, triple consensus'
      #row['Email_1_list'] = ''
      df3.loc[index,('results','list_'+item)] = ''
    else:
      #row['Winner_'+item] = 'N/A'
      df3.loc[index,('results','winner_'+item)] = 'N/A'
      #row[item+'_flag'] = 'All good, triple consensus'
      df3.loc[index,('results','flag_'+item)] = 'All good, triple consensus'
      #row[item+'_list'] = ''
      df3.loc[index,('results','list_'+item)] = ''

### checkpoint

  # Case 2: All 3 are different
  elif (row[item][1] != row[item][2] and row[item][1] != row[item][3] and row[item][2] != row[item][3]):
    templist = []
    #print(row[item][1])
    #print(row[item][2])
    #print(row[item][3])
    #print(type(row[item][3]))

    if (anchor in row[item][1]):
      templist.append(row[item][1])

    if (anchor in row[item][2]):
      templist.append(row[item][2])

    if (anchor in row[item][3]):
      templist.append(row[item][3])

    if len(templist) == 0:
      # row['winner'+item]='N/A'
      df3.loc[index,('results','winner_'+item)] = 'N/A'
      # row['flag_'+item] = 'All good, zero value'
      df3.loc[index,('results','flag_'+item)] = 'All good, zero value'
      # row['list_'+item] = ''
      df3.loc[index,('results','list_'+item)] = ''

    if len(templist) == 1:
      # row['winner'+item]=templist[0]
      df3.loc[index,('results','winner_'+item)] = templist[0]
      # row['flag'+item] = 'Checkit, minority'
      df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'
      # row['list'+item] = ''
      df3.loc[index,('results','list_'+item)] = ''

    if len(templist) == 2:
      # row['winner'+item] = ",".join(templist)
      df3.loc[index,('results','winner_'+item)] = ",".join(templist)
      # row['flag'+item] = 'Checkit, multiple emails'
      df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
      # row['list'+item] = ''
      df3.loc[index,('results','list_'+item)] = ''
  
    if len(templist) == 3:
      df3.loc[index,('results','winner_'+item)] = ",".join(templist)
      df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
      df3.loc[index,('results','list_'+item)] = ''
      
## Checkpoint


  # Case 3: 2 are the same and 1 is different
  #elif row['1'+infix+item] == row['2'+infix+item]: # implicitly, the odd one out is not equal
  elif row[item][1] == row[item][2]: # implicitly, the odd one out is not equal
    # if (anchor in row['1'+infix+item]): # there is an anchor in the overlap
    if (anchor in row[item][1]): # there is an anchor in the overlap
      if (anchor in row[item][3]): # if there is an anchor in the odd one out, we should take a look
        #row['winner'+item] = ",".join([row[item][1],row[item][3]]) #row['1'+infix+item],row['3'+infix+item]])
        df3.loc[index,('results','winner_'+item)] = ",".join([row[item][1],row[item][3]]) #row['1'+infix+item],row['3'+infix+item]])
        # row['flag'+item] = 'Checkit, multiple emails'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      else: # there is no anchor in the odd one out
        # row['winner'+item] = row['1'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][1]
        #row['flag'+item] = 'All good, double consensus'
        df3.loc[index,('results','flag_'+item)] = 'All good, double consensus' 
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''
    
    else: # there is no anchor in the two that overlap
      if anchor in row[item][3]: #row['3'+infix+item]: # if there is an anchor in the odd one out, we should take a look
        # row['winner'+item] = row['3'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][3]
        # row['flag'+item] = 'Checkit, minority'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      else: # there is no anchor in the odd one out or the overlaps
        # row['winner'+item] = 'N/A'
        df3.loc[index,('results','winner_'+item)] = 'N/A'
        # row['flag'+item] = 'All good, zero value'
        df3.loc[index,('results','flag_'+item)] = 'All good, zero value'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''
    # print('hi')
  
## Checkpoint    
  # elif (row['1'+infix+item] == row['3'+infix+item]):
  elif (row[item][1] == row[item][3]):
  
    if (anchor in row[item][1]): # there is an anchor in the overlap
      if (anchor in row[item][2]): # if there is an anchor in the odd one out, we should take a look
        # row['winner'+item] = ','.join([row['1'+infix+item],row['2'+infix+item]])
        df3.loc[index,('results','winner_'+item)] = ','.join([row[item][1],row[item][2]])
        # row['flag'+item] = 'Checkit, multiple emails'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''
      else: # there is no anchor in the odd one out
        # row['winner'+item] = row['1'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][1]
        
        #row['flag'+item] = 'All good, double consensus'
        df3.loc[index,('results','flag_'+item)] = 'All good, double consensus'
        
        #row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

    else: # there is no anchor in the two that overlap
      if anchor in row[item][2]: # if there is an @ in the odd one out, we should take a look
        
        # row['winner'+item] = row['2'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][2]
        
        # row['flag'+item] = 'Checkit, minority'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'
        
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      else: # there is no @ in the odd one out or the overlaps
        # row['winner'+item] = 'N/A'
        df3.loc[index,('results','winner_'+item)] = 'N/A'

        # row['flag'+item] = 'All good, zero value'
        df3.loc[index,('results','flag_'+item)] = 'All good, zero value'

        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

    # print('ho')

## Checkpoint
  #elif (row['2'+infix+item] == row['3'+infix+item]):
  elif (row[item][2] == row[item][3]):
    
    #if (anchor in row['2'+infix+item]): # there is an anchor in the overlap
    if (anchor in row[item][2]): # there is an anchor in the overlap

      #if (anchor in row['1'+infix+item]): # if there is an anchor in the odd one out, we should take a look
      if (anchor in row[item][1]): # if there is an anchor in the odd one out, we should take a look
        #row['winner'+item] = ','.join([row['2'+infix+item],row['1'+infix+item]])
        df3.loc[index,('results','winner_'+item)] = ','.join([row[item][2],row[item][1]])

        #row['flag'+item] = 'Checkit, multiple emails'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'

        #row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      else: # there is no anchor in the odd one out
        # row['winner'+item] = row['2'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][2]
        
        # row['flag'+item] = 'All good, double consensus'
        df3.loc[index,('results','flag_'+item)] = 'All good, double consensus'

        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

## checkpoint    
    else: # there is no anchor in the two that overlap
      #if anchor in row['1'+infix+item]: # if there is an anchor in the odd one out, we should take a look
      if anchor in row[item][1]: #['1'+infix+item]: # if there is an anchor in the odd one out, we should take a look
        #row['winner'+item] = row['1'+infix+item]
        df3.loc[index,('results','winner_'+item)] = row[item][1]

        # row['flag'+item] = 'Checkit, minority'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'

        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      else: # there is no anchor in the odd one out or the overlaps
        #row['winner'+item] = 'N/A'
        df3.loc[index,('results','winner_'+item)] = 'N/A'

        #row['flag'+item] = 'All good, zero value'
        df3.loc[index,('results','flag_'+item)] = 'All good, zero value'

        #row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

df3 = df3.fillna('N/A')


In [ ]:
df3.head()

## Export the dataframe

In [ ]:
df3.to_excel(write_path)